In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader,random_split
from tqdm import tqdm

In [34]:
#Multiple Receptive Field Fusion
class MultipleReceptiveFiledFusion(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels=200, out_channels=50, kernel_size=1, bias=False)
        self.conv2=nn.Conv2d(in_channels=50, out_channels=50, kernel_size=3, dilation=1, stride=1, padding=1, bias=False)
        self.conv3=nn.Conv2d(in_channels=50, out_channels=50, kernel_size=3, dilation=2, stride=1, padding=2, bias=False)
        self.conv4=nn.Conv2d(in_channels=50, out_channels=50, kernel_size=3, dilation=3, stride=1, padding=3, bias=False)
        self.conv5=nn.Conv2d(in_channels=50, out_channels=50, kernel_size=3, dilation=4, stride=1, padding=4, bias=False)
        self.conv6=nn.Conv2d(in_channels=200, out_channels=200, kernel_size=1, stride=1)

    def forward(self, x):
        x1=self.conv1(x)
        print(x1.shape)
        x2_1=self.conv2(x1)
        print(x2_1.shape)
        x2_2=self.conv3(x1)+x2_1
        print(x2_2.shape)
        x2_3=self.conv4(x1)+x2_2
        print(x2_3.shape)
        x2_4=self.conv5(x1)+x2_3
        print(x2_4.shape)
        x3=torch.cat((x2_1,x2_2,x2_3,x2_4),dim=1)
        print(f'x3:{x3.shape},x:{x.shape}')
        x3=self.conv6(x3)+x
        print(f'x3:{x3.shape}')
        return x3

In [35]:
class ResMRFF(nn.Module):
    def __init__(self):
        super().__init__()
        self.dropout = nn.Dropout(0.5)
        self.InstNorm2D=nn.InstanceNorm2d(num_features=200, affine=True)
        self.relu=nn.ReLU()
        self.MRFF=MultipleReceptiveFiledFusion()
    def forward(self, x):
        t=x
        x=self.MRFF(x)
        x=self.dropout(x)
        x=self.InstNorm2D(x)
        x=self.relu(x)
        x=self.MRFF(x)
        x=self.dropout(x)
        x=self.InstNorm2D(x)
        x=self.relu(x)
        return x+t

In [36]:
data=torch.load('IndianPines.pt',weights_only=True)['InputData']
labels=torch.load('IndianPines.pt',weights_only=True)['Label']
data.shape

torch.Size([145, 145, 200])

In [37]:
data = data.unsqueeze(0).permute(0, 3, 1, 2)

In [38]:
net=ResMRFF()
output=net(data)

torch.Size([1, 50, 145, 145])
torch.Size([1, 50, 145, 145])
torch.Size([1, 50, 145, 145])
torch.Size([1, 50, 145, 145])
torch.Size([1, 50, 145, 145])
x3:torch.Size([1, 200, 145, 145]),x:torch.Size([1, 200, 145, 145])
x3:torch.Size([1, 200, 145, 145])
torch.Size([1, 50, 145, 145])
torch.Size([1, 50, 145, 145])
torch.Size([1, 50, 145, 145])
torch.Size([1, 50, 145, 145])
torch.Size([1, 50, 145, 145])
x3:torch.Size([1, 200, 145, 145]),x:torch.Size([1, 200, 145, 145])
x3:torch.Size([1, 200, 145, 145])


In [11]:
class CustomDataset(Dataset):
    def __init__(self, data_list, label_list, transform=None):
        self.data_list = data_list
        self.label_list = label_list
        self.transform = transform

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, index):
        data = self.data_list[index]
        label = self.label_list[index]

        # 如果有转换操作，则应用它
        if self.transform:
            data = self.transform(data)

        return data, label

IndianPines=CustomDataset(data_list=data, label_list=labels)

In [ ]:
# TrainDataSet, ValDataSet=random_split(IndianPines, [len(IndianPines)*0.2, len(IndianPines)*0.8])
#
# TrainDataloader = DataLoader(TrainDataSet, batch_size=32, shuffle=True)
# ValDataloader = DataLoader(ValDataSet, batch_size=32, shuffle=False)
